In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer,text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.callbacks import Callback ,EarlyStopping,ModelCheckpoint
from sklearn.metrics import precision_score, recall_score,accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,GRU, Bidirectional, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
df = pd.read_csv(r'C:\Users\admin\Desktop\Major project\dataset\cleanDataOne.csv')


In [3]:
df['text']

0             cocksucker before you piss around on my work
1        bye do not look come or think of comming back ...
2                   fuck your filthy mother in the ass dry
3        get fucked up get fuckeeed up  got a drink tha...
4        stupid peace of shit stop deleting my stuff as...
                               ...                        
18049    nothing of the sort  sumple and i are merely t...
18050    actually i broke a bone before  twice actually...
18051    ankit fadia study award updated the source for...
18052                           your sources are worthless
18053    this was the first time for me  it was very hi...
Name: text, Length: 18054, dtype: object

In [4]:
df

,text,y
0,cocksucker before you piss around on my work,1
1,bye do not look come or think of comming back ...,1
2,fuck your filthy mother in the ass dry,1
3,get fucked up get fuckeeed up got a drink tha...,1
4,stupid peace of shit stop deleting my stuff as...,1
...,...,...
18049,nothing of the sort sumple and i are merely t...,0
18050,actually i broke a bone before twice actually...,0
18051,ankit fadia study award updated the source for...,0
18052,your sources are worthless,0


In [5]:
X=df['text'].astype(str)
type(X)

pandas.core.series.Series

In [6]:

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
X = pad_sequences(sequences, maxlen=100)

In [8]:
import numpy as np

loaded_arr = np.load(r'C:\Users\admin\Desktop\Major project\embeddingsArray.npy')

# print the loaded array
print(loaded_arr)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.49886     0.76602     0.89750999 ... -0.41179001  0.40538999
   0.78504002]
 [-0.038194   -0.24487001  0.72812003 ... -0.1459      0.82779998
   0.27061999]
 ...
 [ 0.30746001  0.0071693  -0.40382999 ...  0.16892    -0.75593001
  -0.13414   ]
 [ 0.050396   -0.43832001 -0.26798999 ...  0.019003   -0.6778
  -0.30882999]
 [ 0.12239     0.40292999 -0.42792001 ...  0.57059002 -0.23425999
   0.17585   ]]


In [9]:
embedding_matrix=loaded_arr

In [10]:
precision = tf.keras.metrics.Precision(name='precision')
recall = tf.keras.metrics.Recall(name='recall')
early_stop=EarlyStopping(monitor='val_loss',patience=5)
checkpoint = ModelCheckpoint('model_weights.h5', save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min', verbose=1)


In [12]:
modelGRUBidirectionalGlove = Sequential()
modelGRUBidirectionalGlove.add(Embedding(input_dim=26611, output_dim=100,weights=[embedding_matrix],trainable=False))

modelGRUBidirectionalGlove.add(Bidirectional(GRU(64)))
modelGRUBidirectionalGlove.add(Dense(1, activation='sigmoid'))
modelGRUBidirectionalGlove.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',precision,recall])




In [13]:
x_train, x_val, y_train, y_val = train_test_split(X, df['y'], test_size=0.2, random_state=42)

In [14]:
history=modelGRUBidirectionalGlove.fit(x_train, y_train,validation_data=(x_val,y_val), epochs=30,batch_size=32,callbacks=[early_stop,checkpoint])


Epoch 1/30
452/452 [==============================] - ETA: 0s - loss: 0.3356 - accuracy: 0.8574 - precision: 0.8669 - recall: 0.8445
Epoch 1: val_loss improved from inf to 0.27309, saving model to model_weights.h5
452/452 [==============================] - 24s 46ms/step - loss: 0.3356 - accuracy: 0.8574 - precision: 0.8669 - recall: 0.8445 - val_loss: 0.2731 - val_accuracy: 0.8851 - val_precision: 0.8986 - val_recall: 0.8682
Epoch 2/30
451/452 [============================>.] - ETA: 0s - loss: 0.2566 - accuracy: 0.8964 - precision: 0.9027 - recall: 0.8886
Epoch 2: val_loss improved from 0.27309 to 0.26259, saving model to model_weights.h5
452/452 [==============================] - 21s 47ms/step - loss: 0.2566 - accuracy: 0.8965 - precision: 0.9028 - recall: 0.8887 - val_loss: 0.2626 - val_accuracy: 0.8914 - val_precision: 0.8675 - val_recall: 0.9241
Epoch 3/30
451/452 [============================>.] - ETA: 0s - loss: 0.2350 - accuracy: 0.9069 - precision: 0.9112 - recall: 0.9017
Epoch

In [15]:
gru_bidirectional_Glove_pickle_data={'data':history.history,'model':modelGRUBidirectionalGlove}

In [16]:
import pickle

# Save the trained model object to a file
with open(r'C:\Users\admin\Desktop\Major project\dataset\modelGRUBidirectionalGlove.pkl', 'wb') as f:
    pickle.dump(gru_bidirectional_Glove_pickle_data, f)


Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\bidirectional
......vars
...layers\bidirectional\backward_layer
......vars
...layers\bidirectional\backward_layer\cell
......vars
.........0
.........1
.........2
...layers\bidirectional\forward_layer
......vars
...layers\bidirectional\forward_layer\cell
......vars
.........0
.........1
.........2
...layers\bidirectional\layer
......vars
...layers\bidirectional\layer\cell
......vars
...layers\dense
......vars
.........0
.........1
...layers\embedding
......vars
.........0
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...metrics\precision
......vars
.........0
.........1
...metrics\recall
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Ker